# Mais sobre `SELECT`

- temporary tables
- views
- variables
- subqueries


In [1]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="sakila"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)

db = partial(run_db_query, connection)

## Exercícios para entrega

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [2]:
ia.tasks()

|    | Atividade    | De                        | Até                       |
|---:|:-------------|:--------------------------|:--------------------------|
|  0 | newborn      | 2024-02-01 03:00:00+00:00 | 2024-05-30 03:00:00+00:00 |
|  1 | select01     | 2024-02-08 03:00:00+00:00 | 2024-02-19 02:59:59+00:00 |
|  2 | ddl          | 2024-02-22 03:00:00+00:00 | 2024-02-27 02:59:59+00:00 |
|  3 | dml          | 2024-02-26 03:00:00+00:00 | 2024-03-03 02:59:59+00:00 |
|  4 | views        | 2024-02-29 03:00:00+00:00 | 2024-03-16 02:59:59+00:00 |
|  5 | group_having | 2024-02-29 03:00:00+00:00 | 2024-03-08 03:00:00+00:00 |
|  6 | agg_join     | 2024-02-29 03:00:00+00:00 | 2024-03-05 02:59:59+00:00 |

In [3]:
ia.grades(by="task")

|    | Tarefa       |   Nota |
|---:|:-------------|-------:|
|  0 | agg_join     |   10   |
|  1 | ddl          |   10   |
|  2 | dml          |    0   |
|  3 | group_having |    0   |
|  4 | newborn      |   10   |
|  5 | select01     |   10   |
|  6 | views        |    2.5 |

In [4]:
ia.grades(task="views")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | views       | ex01        |      1 |     10 |
|  1 | views       | ex02        |      1 |     10 |
|  2 | views       | ex03        |      1 |     10 |
|  3 | views       | ex04        |      1 |      0 |
|  4 | views       | ex05        |      1 |      0 |
|  5 | views       | ex06        |      1 |      0 |
|  6 | views       | ex07        |      1 |      0 |
|  7 | views       | ex08        |      1 |      0 |
|  8 | views       | ex09        |      1 |      0 |
|  9 | views       | ex10        |      1 |      0 |
| 10 | views       | ex11        |      1 |      0 |
| 11 | views       | ex12        |      1 |      0 |

## Aquecimento

Quanta receita foi gerada para cada categoria de filmes? Mostre o nome da categoria e a receita. Ordene da maior receita para para a menor.

In [5]:
sql_ex01 = """
SELECT category.name AS Categoria, 
SUM(payment.amount) AS Receita
FROM category
JOIN film_category ON category.category_id = film_category.category_id
JOIN film ON film_category.film_id = film.film_id
JOIN inventory ON film.film_id = inventory.film_id
JOIN rental ON inventory.inventory_id = rental.inventory_id
JOIN payment ON rental.rental_id = payment.rental_id
GROUP BY category.name
ORDER BY Receita DESC;
"""


db(sql_ex01)

Executando query:
('Sports', Decimal('5314.21'))
('Sci-Fi', Decimal('4756.98'))
('Animation', Decimal('4656.30'))
('Drama', Decimal('4587.39'))
('Comedy', Decimal('4383.58'))
('Action', Decimal('4375.85'))
('New', Decimal('4351.62'))
('Games', Decimal('4281.33'))
('Foreign', Decimal('4270.67'))
('Family', Decimal('4226.07'))
('Documentary', Decimal('4217.52'))
('Horror', Decimal('3722.54'))
('Children', Decimal('3655.55'))
('Classics', Decimal('3639.59'))
('Travel', Decimal('3549.64'))
('Music', Decimal('3417.72'))


In [6]:
ia.sender(answer="sql_ex01", task="views", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

Liste os 10 filmes mais alugados e o numero de vezes em que foram alugados. Ordene do mais alugado para o menos alugado.

In [7]:
sql_ex02 = """
SELECT film.title AS Titulo, 
COUNT(rental.rental_id) AS Numero_de_Alugueis
FROM film
JOIN inventory ON film.film_id = inventory.film_id
JOIN rental ON inventory.inventory_id = rental.inventory_id
GROUP BY film.title
ORDER BY Numero_de_Alugueis DESC
LIMIT 10;

"""

db(sql_ex02)

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('RIDGEMONT SUBMARINE', 32)
('GRIT CLOCKWORK', 32)
('SCALAWAG DUCK', 32)
('JUGGLER HARDLY', 32)
('FORWARD TEMPLE', 32)
('HOBBIT ALIEN', 31)
('ROBBERS JOON', 31)
('ZORRO ARK', 31)


In [8]:
ia.sender(answer="sql_ex02", task="views", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

**DESAFIO:** Liste os filmes e o numero de vezes em que foram alugados apenas para filmes que foram alugados mais do que a média de numero de alugueis por filme. *Dica*: Serão necessários múltiplos SELECTs. Faça cada um em uma célula diferente.

In [9]:
db("""
SELECT AVG(alugueis_por_filme.Numero_de_Alugueis) AS Media_De_Alugueis
FROM (
    SELECT COUNT(rental.rental_id) AS Numero_de_Alugueis
    FROM rental
    JOIN inventory ON rental.inventory_id = inventory.inventory_id
    GROUP BY inventory.film_id
) AS alugueis_por_filme;
""")
db("""
SELECT film.film_id, film.title AS Titulo, 
COUNT(rental.rental_id) AS Numero_de_Alugueis
FROM film
JOIN inventory ON film.film_id = inventory.film_id
JOIN rental ON inventory.inventory_id = rental.inventory_id
GROUP BY film.film_id, film.title;

""")
db("""
SELECT f.title AS Titulo, 
COUNT(r.rental_id) AS Numero_de_Alugueis
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.title
HAVING COUNT(r.rental_id) > (
    SELECT AVG(alugueis_por_filme.Numero_de_Alugueis) AS Media_De_Alugueis
    FROM (
        SELECT COUNT(rental.rental_id) AS Numero_de_Alugueis
        FROM rental
        JOIN inventory ON rental.inventory_id = inventory.inventory_id
        GROUP BY inventory.film_id
    ) AS alugueis_por_filme
)
ORDER BY Numero_de_Alugueis DESC;

""")

Executando query:
(Decimal('16.7474'),)
Executando query:
(1, 'ACADEMY DINOSAUR', 23)
(2, 'ACE GOLDFINGER', 7)
(3, 'ADAPTATION HOLES', 12)
(4, 'AFFAIR PREJUDICE', 23)
(5, 'AFRICAN EGG', 12)
(6, 'AGENT TRUMAN', 21)
(7, 'AIRPLANE SIERRA', 15)
(8, 'AIRPORT POLLOCK', 18)
(9, 'ALABAMA DEVIL', 12)
(10, 'ALADDIN CALENDAR', 23)
(11, 'ALAMO VIDEOTAPE', 24)
(12, 'ALASKA PHANTOM', 26)
(13, 'ALI FOREVER', 9)
(15, 'ALIEN CENTER', 22)
(16, 'ALLEY EVOLUTION', 14)
(17, 'ALONE TRIP', 18)
(18, 'ALTER VICTORY', 22)
(19, 'AMADEUS HOLY', 21)
(20, 'AMELIE HELLFIGHTERS', 10)
(21, 'AMERICAN CIRCUS', 22)
(22, 'AMISTAD MIDSUMMER', 21)
(23, 'ANACONDA CONFESSIONS', 21)
(24, 'ANALYZE HOOSIERS', 14)
(25, 'ANGELS LIFE', 22)
(26, 'ANNIE IDENTITY', 22)
(27, 'ANONYMOUS HUMAN', 13)
(28, 'ANTHEM LUKE', 15)
(29, 'ANTITRUST TOMATOES', 10)
(30, 'ANYTHING SAVANNAH', 9)
(31, 'APACHE DIVINE', 31)
(32, 'APOCALYPSE FLAMINGOS', 6)
(34, 'ARABIA DOGMA', 13)
(35, 'ARACHNOPHOBIA ROLLERCOASTER', 24)
(37, 'ARIZONA BANG', 21)
(39, 'ARMA

## Views

Uma *view* é uma tabela virtual, construida a partir de um comando `SELECT`. Por exemplo: execute o código a seguir.

In [10]:
db("""
DROP VIEW IF EXISTS movie_count;
""")

db("""
CREATE VIEW movie_count AS
    SELECT 
        title, COUNT(rental_id) as cnt
    FROM
        film
        LEFT OUTER JOIN inventory USING (film_id)
        LEFT OUTER JOIN rental USING (inventory_id)
    GROUP BY
        film_id
    ORDER BY
        cnt ASC;
""")

Executando query:
Executando query:


Agora temos uma *view* chamada `movie_count`. Vamos verificar que ela funcionou, listando as 30 primeiras linhas:

In [11]:
db("""
SELECT 
    *
FROM
    movie_count 
LIMIT 
    30
""")

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


In [12]:
db("""
SELECT 
    * 
FROM 
    movie_count 
ORDER BY
    cnt DESC
LIMIT 30
""")

Executando query:
('BUCKET BROTHERHOOD', 34)
('ROCKETEER MOTHER', 33)
('SCALAWAG DUCK', 32)
('RIDGEMONT SUBMARINE', 32)
('JUGGLER HARDLY', 32)
('GRIT CLOCKWORK', 32)
('FORWARD TEMPLE', 32)
('ZORRO ARK', 31)
('WIFE TURN', 31)
('TIMBERLAND SKY', 31)
('RUSH GOODFELLAS', 31)
('ROBBERS JOON', 31)
('NETWORK PEAK', 31)
('HOBBIT ALIEN', 31)
('GOODFELLAS SALUTE', 31)
('APACHE DIVINE', 31)
('WITCHES PANIC', 30)
('SUSPECTS QUILLS', 30)
('SHOCK CABIN', 30)
('RUGRATS SHAKESPEARE', 30)
('PULP BEVERLY', 30)
('MUSCLE BRIGHT', 30)
('MASSACRE USUAL', 30)
('MARRIED GO', 30)
('IDOLS SNATCHERS', 30)
('HARRY IDAHO', 30)
('GRAFFITI LOVE', 30)
('FROST HEAD', 30)
('ENGLISH BULWORTH', 30)
('DOGMA FAMILY', 30)


Agora suponha que alteramos a tabela `film`, mudando o nome do filme "DAZED PUNK" para "STONED PUNK".

**Atividade**: Do it.

In [13]:
db("""
-- SUA QUERY AQUI!
""")

Executando query:


Verifique agora a nossa *view*:

In [14]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


Como você pode ver, as views são tabelas virtuais que são **automaticamente atualizadas quando as tabelas originais são modificadas**.

Sempre que você realizar modificações nos dados, dê `commit` ou `rollback`. Ainda, evite executar múltiplas vezes as linhas de código que criem a conexão sem antes ter fechado a conexão ativa.

Vamos desfazer as alterações:

In [15]:
connection.rollback()

Conferindo:

In [16]:
db("""
SELECT
    *
FROM
    movie_count
LIMIT
    30
""")

Executando query:
('ALICE FANTASIA', 0)
('APOLLO TEEN', 0)
('ARGONAUTS TOWN', 0)
('ARK RIDGEMONT', 0)
('ARSENIC INDEPENDENCE', 0)
('BOONDOCK BALLROOM', 0)
('BUTCH PANTHER', 0)
('CATCH AMISTAD', 0)
('CHINATOWN GLADIATOR', 0)
('CHOCOLATE DUCK', 0)
('COMMANDMENTS EXPRESS', 0)
('CROSSING DIVORCE', 0)
('CROWDS TELEMARK', 0)
('CRYSTAL BREAKING', 0)
('DAZED PUNK', 0)
('DELIVERANCE MULHOLLAND', 0)
('FIREHOUSE VIETNAM', 0)
('FLOATS GARDEN', 0)
('FRANKENSTEIN STRANGER', 0)
('GLADIATOR WESTWARD', 0)
('GUMP DATE', 0)
('HATE HANDICAP', 0)
('HOCUS FRIDA', 0)
('KENTUCKIAN GIANT', 0)
('KILL BROTHERHOOD', 0)
('MUPPET MILE', 0)
('ORDER BETRAYED', 0)
('PEARL DESTINY', 0)
('PERDITION FARGO', 0)
('PSYCHO SHRUNK', 0)


### Vamos praticar

Verifique quantas vezes o filme "COWBOY DOOM" foi alugado usando a view `movie_count`

In [17]:
sql_ex03 = '''
SELECT movie_count.cnt
FROM movie_count
WHERE movie_count.title = 'COWBOY DOOM';
'''


db(sql_ex03)

Executando query:


(7,)


In [45]:
ia.sender(answer="sql_ex03", task="views", question="ex03", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

Registre um aluguel do filme "COWBOY DOOM" (com o menor id possível), feito pelo funcionario "Jon Stephens" na loja id=1 para o cliente "JESSIE BANKS", na data '2019-01-01', com data de retorno '2019-01-08'.

In [46]:
sql_ex04 = """
SELECT film_id 
FROM film 
WHERE title = 'COWBOY DOOM';

SELECT customer_id 
FROM customer 
WHERE first_name = 'JESSIE' AND last_name = 'BANKS';

SELECT staff_id 
FROM staff 
WHERE first_name = 'Jon' AND last_name = 'Stephens';

SELECT inventory_id 
FROM inventory 
JOIN film ON inventory.film_id = film.film_id 
WHERE film.title = 'COWBOY DOOM' AND inventory.store_id = 1 
ORDER BY inventory_id ASC LIMIT 1;

"""

db(sql_ex04)

OperationalError: MySQL Connection not available.

In [41]:
db('''
INSERT INTO rental (rental_date, inventory_id, customer_id, return_date, staff_id)
VALUES ('2019-01-01', [inventory_id_obtido], [customer_id_obtido], '2019-01-08', [staff_id_obtido]);
''')

OperationalError: MySQL Connection not available.

In [21]:
ia.sender(answer="sql_ex04", task="views", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

Verifique usando a view que a contagem de alugueis do filme subiu.

In [22]:
db("""
-- SUA QUERY AQUI!
""")

Executando query:


Faça o *rollback* desta alteração, para não modificar a nossa querida base de dados *sakila*.

In [23]:
connection.rollback()

## Tabelas temporárias

Tabelas temporárias podem ser criadas para ajudar nas tarefas de manipulação de dados. Essas tabelas existem apenas pela duração da sessão. Para criar uma tabela temporária, basta adicionar a palavra-chave `TEMPORARY` no momento da criação.

É comum criar tabelas temporárias à partir do resultado de comandos `SELECT`. Por exemplo, a seguinte query permite montar uma tabela temporária com os filmes que duram mais que 3 horas:

In [24]:
db("""
DROP TABLE IF EXISTS long_film
""")

db("""
CREATE TEMPORARY TABLE long_film 
    SELECT
        *
    FROM
        film
    WHERE
        film.length > 180;
""")

Executando query:
Executando query:


Podemos verificar que a tabela `long_film` agora existe:

In [25]:
db("DESCRIBE long_film")

Executando query:
('film_id', 'smallint unsigned', 'NO', '', '0', 'NULL')
('title', 'varchar(128)', 'NO', '', None, 'NULL')
('description', 'text', 'YES', '', None, 'NULL')
('release_year', 'year', 'YES', '', None, 'NULL')
('language_id', 'tinyint unsigned', 'NO', '', None, 'NULL')
('original_language_id', 'tinyint unsigned', 'YES', '', None, 'NULL')
('rental_duration', 'tinyint unsigned', 'NO', '', '3', 'NULL')
('rental_rate', 'decimal(4,2)', 'NO', '', '4.99', 'NULL')
('length', 'smallint unsigned', 'YES', '', None, 'NULL')
('replacement_cost', 'decimal(5,2)', 'NO', '', '19.99', 'NULL')
('rating', "enum('G','PG','PG-13','R','NC-17')", 'YES', '', 'G', 'NULL')
('special_features', "set('Trailers','Commentaries','Deleted Scenes','Behind the Scenes')", 'YES', '', None, 'NULL')
('last_update', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', 'on update CURRENT_TIMESTAMP')


In [26]:
db("CALL sys.table_exists('sakila', 'long_film', @table_type);")
db("SELECT @table_type;")

Executando query:
Executando query:
('TEMPORARY',)


Muito embora ela não apareça na lista de tabelas: isso é um bug do MySQL. (https://dev.mysql.com/worklog/task/?id=648)

In [27]:
db("SHOW TABLES")

Executando query:
('actor',)
('actor_info',)
('address',)
('category',)
('city',)
('country',)
('customer',)
('customer_list',)
('film',)
('film_actor',)
('film_category',)
('film_list',)
('film_text',)
('inventory',)
('language',)
('movie_count',)
('nicer_but_slower_film_list',)
('payment',)
('rental',)
('sales_by_film_category',)
('sales_by_store',)
('staff',)
('staff_list',)
('store',)


Vamos listar o conteudo desta tabela:

In [28]:
db("SELECT title FROM long_film")

Executando query:
('ANALYZE HOOSIERS',)
('BAKED CLEOPATRA',)
('CATCH AMISTAD',)
('CHICAGO NORTH',)
('CONSPIRACY SPIRIT',)
('CONTROL ANTHEM',)
('CRYSTAL BREAKING',)
('DARN FORRESTER',)
('FRONTIER CABIN',)
('GANGS PRIDE',)
('HAUNTING PIANIST',)
('HOME PITY',)
('HOTEL HAPPINESS',)
('INTRIGUE WORST',)
('JACKET FRISCO',)
('KING EVOLUTION',)
('LAWLESS VISION',)
('LOVE SUICIDES',)
('MONSOON CAUSE',)
('MOONWALKER FOOL',)
('MUSCLE BRIGHT',)
('POND SEATTLE',)
('RECORDS ZORRO',)
('REDS POCUS',)
('RUNAWAY TENENBAUMS',)
('SATURN NAME',)
('SCALAWAG DUCK',)
('SEARCHERS WAIT',)
('SMOOCHY CONTROL',)
('SOLDIERS EVOLUTION',)
('SONS INTERVIEW',)
('SORORITY QUEEN',)
('STAR OPERATION',)
('SWEET BROTHERHOOD',)
('THEORY MERMAID',)
('WIFE TURN',)
('WILD APOLLO',)
('WORST BANGER',)
('YOUNG LANGUAGE',)


Vamos apagar a tabela `long_film`:

In [29]:
db("DROP TABLE long_film")

Executando query:


### Vamos praticar

- Crie uma tabela temporária `max_duration` que contém a duração máxima de filme para cada categoria. Apresente o id da categoria, seu nome e a duração máxima.

In [30]:
# Executamos o DROP apenas localmente, sem enviar ao servidor
db("DROP TABLE IF EXISTS max_duration")

sql_ex05 = """
-- SUA QUERY AQUI!
"""

db(sql_ex05)

Executando query:
Executando query:


In [31]:
ia.sender(answer="sql_ex05", task="views", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

 - Verifique a tabela.

In [47]:
db("SELECT * FROM max_duration")

OperationalError: MySQL Connection not available.

In [ ]:
db("DESCRIBE max_duration")

- Agora use a tabela temporária para construir uma consulta com as categorias e seus respectivos filmes mais longos:

In [ ]:
db("""
SELECT
    category_id, name, film_id, title, length
FROM
    film
    INNER JOIN film_category USING (film_id)
    INNER JOIN max_duration USING (category_id)
WHERE
    length = max_len
""")

- delete a tabela temporária

In [ ]:
db("""
DROP TABLE max_duration
""")

## Variáveis

Podemos montar uma query que retorne um valor só e armazenar este valor em uma variável, para uso posterior em outras queries. Para isso vamos usar o prefixo '@' para indicar variáveis, e o comando `SELECT ... INTO`.

Exemplo: quais são os filmes "caros" da nossa base sakila? Vamos descobrir quais filmes custam mais que um desvio padrão acima da média de preços de locação.

Primeiro vamos calcular a média e o desvio padrão dos preços de aluguel:

In [ ]:
db("""
SELECT 
    AVG(rental_rate), 
    STDDEV(rental_rate)
INTO 
    @avg_rate, 
    @stddev_rate 
FROM
    film;
""")

Note que a query não retorna um resultado: o resultado foi armazenado direto nas variáveis `@avg_rate` e `@stddev_rate`. Vamos usar um `SELECT` sem tabelas para ver o resultado:

In [ ]:
db("SELECT @avg_rate, @stddev_rate")

Agora podemos selecionar os filmes caros!

In [ ]:
db("""
SELECT 
    title, rental_rate
FROM
    film
WHERE
    rental_rate > @avg_rate + @stddev_rate
LIMIT 10
""")

### Vamos praticar

Armazene na variável temporária `max_films` a quantidade de filmes feitos pelo ator ou atriz que mais participou de filmes.

In [ ]:
sql_ex06 = """
-- SUA QUERY AQUI!
"""

db(sql_ex06)

In [ ]:
db("""
SELECT @max_films
""")

In [ ]:
ia.sender(answer="sql_ex06", task="views", question="ex06", answer_type="pyvar")

## Operador `IN`

Suponha que desejamos listar todos os filmes dos 3 atores mais populares. Podemos começar listando os 3 atores mais populares:

In [ ]:
db("""
SELECT 
    actor_id, first_name, last_name, COUNT(film_id) AS num_films
FROM
    actor
    INNER JOIN film_actor USING (actor_id)
GROUP BY 
    actor_id
ORDER BY 
    num_films DESC
LIMIT 3
""")

Vamos criar uma tabela temporária para guardar a informação de `actor_id` desses atores:

In [ ]:
db("DROP TABLE IF EXISTS temp_pop_actors")
db("""
CREATE TEMPORARY TABLE temp_pop_actors
    SELECT first_name, last_name, actor_id FROM
        actor
        INNER JOIN film_actor USING (actor_id)
    GROUP BY 
        actor_id
    ORDER BY 
        COUNT(film_id) DESC
    LIMIT 3
""")
db("""
SELECT * from temp_pop_actors
""")

Por fim, vamos usar essa informação para listar os filmes dos atores populares:

In [ ]:
db("""
SELECT DISTINCT
    title
FROM
    film
    INNER JOIN film_actor USING (film_id)
WHERE
    actor_id IN (SELECT actor_id FROM temp_pop_actors);
""")

Note o uso de *subqueries*!

Não se esqueça de limpar tudo no final!

In [ ]:
db("DROP TABLE temp_pop_actors")

### Vamos praticar

Liste os atores (id, nome e sobrenome) que participaram dos 3 filmes mais rentáveis (aqueles que mais geraram receita para a locadora) ordenados pelo id do ator de modo crescente. Para isso, crie uma tabela temporária contendo o id do filme e a quantia total e use essa tabela para listar os atores.

In [ ]:
db("""
-- SUA QUERY AQUI!
""")
db("""
-- SUA QUERY AQUI!
""")

In [ ]:
sql_ex07 = ["""
-- SUA QUERY AQUI!
""",
"""
-- SUA QUERY AQUI!
"""]

ia.sender(answer="sql_ex07", task="views", question="ex07", answer_type="pyvar")

## Subqueries

Os tópicos discutidos acima poderiam ser resolvidos, em grande parte, com subqueries. As subqueries são queries `SELECT` criadas dentro de outras queries. 

Poderíamos ter usado subqueries nos mesmos lugares onde usamos tabelas temporárias, nos tópicos acima. Quando a subquery pode ser transformada em uma tabela temporária independente, separada da query exterior, dizemos que a subquery é **não-correlacionada** com a query exterior.

Usar subqueries não-correlacionadas é um tópico controverso: podemos sempre usar uma tabela temporária ou, ás vezes, pensar em um `JOIN` simples. Aliás, muitas vezes o otimizador de queries do banco de dados transformará a subquery em `JOIN`, se isso for vantajoso em termos de desempenho.

Uma subquery que depende da query externa (e portanto não pode ser separada em uma tabela temporária independente) é chamada de **subquery correlacionada**. Nestes casos podemos ter que executar a subquery para cada linha da query exterior! 

### Vamos praticar

Vamos refazer a atividade dos atores dos filmes mais rentáveis, usando subqueries. 

Temos um problema: o MySQL não suporta ``LIMIT`` em subqueries com o operador ``IN``. Vamos investigar isso mais de perto. 

Em primeiro lugar faça uma tradução direta da implementação da atividade anterior trocando tabela temporária por subquery.

In [ ]:
try:
    db('''
-- SUA QUERY AQUI!
    ''')
except mysql.connector.ProgrammingError as e:
    print(f'ProgrammingError: {e}')

Ok, apareceu o problema. Mas considere que o problema original não precisava de IN desde o começo! Construa essa solução.

In [ ]:
sql_ex08 = """
-- SUA QUERY AQUI!
"""

db(sql_ex08)

In [ ]:
ia.sender(answer="sql_ex08", task="views", question="ex08", answer_type="pyvar")

# `UNION`

Quando duas tabelas tem **EXATAMENTE** as mesmas colunas, podemos concatená-las e formar uma grande tabela unificada usando o operador `UNION`. Por exemplo: suponha que desejamos montar uma lista dos nomes e sobrenomes de todos os clientes E de todos os funcionários. Eis uma solução possível:

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("""
CREATE TEMPORARY TABLE nomes_clientes 
    SELECT first_name, last_name FROM customer
""")

In [ ]:
db("DESCRIBE nomes_clientes")
db("SELECT * FROM nomes_clientes LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_staff")
db("""
CREATE TEMPORARY TABLE nomes_staff 
    SELECT first_name, last_name FROM staff
""")

In [ ]:
db("DESCRIBE nomes_staff")
db("SELECT * FROM nomes_staff LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_all")
db("""
CREATE TEMPORARY TABLE nomes_all 
    (SELECT * FROM nomes_staff)
    UNION 
    (SELECT * FROM nomes_clientes)
""")

In [ ]:
db("DESCRIBE nomes_all")
db("SELECT * FROM nomes_all LIMIT 5")

In [ ]:
db("DROP TABLE IF EXISTS nomes_clientes")
db("DROP TABLE IF EXISTS nomes_staff")
db("DROP TABLE IF EXISTS nomes_all")

**Vamos praticar:** refaça o exemplo acima mas use *subqueries* ao invés de *temp tables*. Ordene de forma ascendente por first_name e last_name.

In [ ]:
sql_ex09 = """
-- SUA QUERY AQUI!
"""

db(sql_ex09)

In [ ]:
ia.sender(answer="sql_ex09", task="views", question="ex09", answer_type="pyvar")

## Desafios!

Faça uma lista de filmes que tenham mais de dois atores cujo nome inicia com a mesma letra do título do filme! Apresente o nome e a quantidade de atores. Ordene pelo id do filme. Dica: Pesquie a função `LEFT`

In [ ]:
sql_ex10 = """
-- SUA QUERY AQUI!
"""
db(sql_ex10)

In [ ]:
ia.sender(answer="sql_ex10", task="views", question="ex10", answer_type="pyvar")

Semana do "DAN HARRIS": liste os clientes (nome e sobrenome) que nunca assistiram um filme do ator "DAN HARRIS" ou que já assistiram mas a ultima vez em que assistiram um filme dele foi antes de '2005-06-01'. Ordene pelo nome e sobrenome do cliente.

Considerem que a pessoa pode ter assistido um filme de Dan Harris se:

- Devolveu em 2005-06-01 ou depois
- Alugou em 2005-06-01 ou depois
- Alugou antes de 2005-06-01 e ainda não devolveu

In [ ]:
sql_ex11 = """
-- SUA QUERY AQUI!
"""
db(sql_ex11)

In [ ]:
ia.sender(answer="sql_ex11", task="views", question="ex11", answer_type="pyvar")

- Faça uma consulta que retorne, para cada ator, a seguinte informação:

| first_name | last_name | filmes por categoria |
|--|--|--|
| PENELOPE | GUINESS | Animation: ANACONDA CONFESSIONS; Children: LANGUAGE COWBOY; Classics: COLOR PHILADELPHIA, WESTWARD SEABISCUIT; Comedy: VERTIGO NORTHWEST; Documentary: ACADEMY DINOSAUR; Family: KING EVOLUTION, SPLASH GUMP; Foreign: MULHOLLAND BEAST; Games: BULWORTH COMMANDMENTS, HUMAN GRAFFITI; Horror: ELEPHANT TROJAN, LADY STAGE, RULES HUMAN; Music: WIZARD COLDBLOODED; New: ANGELS LIFE, OKLAHOMA JUMANJI; Sci-Fi: CHEAPER CLYDE; Sports: GLEAMING JAWBREAKER |
| NICK | WAHLBERG | Action: BULL SHAWSHANK; Animation: FIGHT JAWBREAKER; Children: JERSEY SASSY; Classics: DRACULA CRYSTAL, GILBERT PELICAN; Comedy: MALLRATS UNITED, RUSHMORE MERMAID; Documentary: ADAPTATION HOLES; Drama: WARDROBE PHANTOM; Family: APACHE DIVINE, CHISUM BEHAVIOR, INDIAN LOVE, MAGUIRE APACHE; Foreign: BABY HALL, HAPPINESS UNITED; Games: ROOF CHAMPION; Music: LUCKY FLYING; New: DESTINY SATURDAY, FLASH WARS, JEKYLL FROGMEN, MASK PEACH; Sci-Fi: CHAINSAW UPTOWN, GOODFELLAS SALUTE; Travel: LIAISONS SWEET, SMILE EARRING |
| etc | etc | etc |

Ordene pelo nome e sobrenome do ator.

Dica: use `GROUP_CONCAT` para agrupar todas as strings de uma coluna em uma string só, e `CONCAT` para unir strings particulares.

In [ ]:
sql_ex12 = """
-- SUA QUERY AQUI!
"""

db(sql_ex12)

In [ ]:
ia.sender(answer="sql_ex12", task="views", question="ex12", answer_type="pyvar")

## Conclusão

Façamos uma pausa para apreciar quão longe estamos: já conseguimos criar nossas tabelas, inserir informação, removê-la, atualizá-la, e consultar nossa base de maneiras bem sofisticadas! Vimos desde `SELECT` simples até buscas mais complexas envolvendo várias etapas de processamento para obter o dado desejado.

Por hoje é só, feche sua conexão e bom descanso!

In [ ]:
connection.close()

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [ ]:
ia.grades(by="TASK", task="views")

In [ ]:
ia.grades(task="views")

In [ ]:
ia.grades(by="task")